# Calculating Copper - Jack Maughan

The ExploreSA Gawler Challenge was designed to find and apply new approaches to mineral exploration, using state geology and geophysics data.  The following notebook is a part of the series, __'Calculating Copper'__ and contains my thinking, processing and results for the challenge, with a idea of recreating a reproducible workflow for similar mineral exploration problems. 

![Title](https://docs.google.com/uc?export=download&id=1goFsreNqBipsuyFEOwi6cGgrpWwARCEt)

#### 1. Geology Forever - Background Information
#### 2. Hair and Makeup - Target Variable and Features
#### 3. Suit up - Compiling Dataframe and Feature Statistics
#### 4. My Friend Robot - Machine Learning Model Creation
### *5. Gold Rush - Model Validation and Results* 
#### 6. Extra Extra - Unsupervised Learning and CoDa on Soil Geochemistry

# Gold Rush

In the previous notebook in this series, __My Friend Robot__, we created and deployed 8 different machine learning models and 1 deep learning model on our Gawler Craton study area. In this notebook, we use the locations of known/existing mineral deposits to validate how well our models perform. Firstly, we will import the required python libraries and look at each of the model results over the entire craton. Note that instead of using the direct model results (either 0 for barren or 1 for mineralised), we will use probability of each 250m x 250m cell being mineralised on a scale of 0 to 1.

In [1]:
import pandas as pd
import glob
import os
from IPython.display import Image, display
from ipywidgets import interact, interact_manual

pd.options.display.max_columns = None

In [2]:
@interact
def show_images(file=os.listdir('Results/')):
    display(Image('Results/'+file))

interactive(children=(Dropdown(description='file', options=('Convolutional Neural Network.png', 'XGBoost.png',…

__NOTE: If the above interactive image viewer does not work, all prediction maps can be found [here](https://drive.google.com/drive/folders/1j8OZ4H7tzHhTkuA-sLr25NzFKgYCa2vw?usp=sharing)__

The different models show similar areas of predicted mineralisation, following a similar trend to the IOCG exploration model seen in __Geology Forever__. We saw in the __Suit Up__ notebook that our target variable was most dependent on the AusLAMP magnetotelluric data and the distance to faults feature. This is evident in our model results, where the areas of higher mineralisation probability are mainly represented by the distance to faults feature (particularly noticeable in Logistic Regression and Linear Discrimination Analysis) or the mangetotellurics (particularly noticeable in the Random Forest, Light GBM and XGBoost models).

## Validation

To validate how well the models perform, we will compare the areas of high mineralisation probability to areas where we know existing mines/deposits are (from the existing mines and deposits data on SARIG). It is important to keep in mind that the drillholes with economic mineralisation that were used for defining our areas of interests will more than likely coincide with existing/known deposits. We will go through each of the model results from bad to good and view some of the interesting results.

### Logistic Regression, Linear Discriminant Analysis and Naive Bayes

As seen in the images above, the results from the logistic regression, linear discriminant analysis and naive bayes models are strongly correlated with the faults feature and don't show much correlation with the existing mines. This lines up with the relatively poor model metrics for these classifiers as well. Without opening an investigation into why these models don't perform as well, and if there's anything we can do to increase their performance, we will exclude these three models from any further interpretations.

### Support Vector Machine

Similarly to the three models above, the support vector machine is strongly correlated with the distance to faults feature for the majority of the craton, but does show some interesting relationships with our known deposits. For the SVM, we filtered the results to only include areas with at least a 75% probability of being mineralised. There are two interesting areas to report.

![Title](https://docs.google.com/uc?export=download&id=14yUuOicBprQka9uHZSfaNRxFKsR5XpSy)

In the map above, we can see the Challenger Gold mine region and the surrounding gold prospects. The Red and Green diamonds are reflective of areas used in model creation and the circle points are the existing deposits/mines. This map is interesting because it shows that the gold deposits in this area are all located exactly in areas where the model says there is low mineralisation (good start). However, after some research the majority of these deposits (if not all) contain economic gold that has been redistributed/remobilised via regional metamorphism and partial melting of the pelitic host rocks. This tells us that the source for these deposits are not our targeted hydrothermal/magmatic sources, which could lead to some errors in this area.

![Title](https://docs.google.com/uc?export=download&id=1OuJQ5egNbiTZlrmtiHodfsTMcPbLsZHr)

Another interesting area of the SVM model is around the Greenwood, Mainwood and Campfire Bore deposits. Here we can see that even though the data fed into the model said the Mainwood area was not of interest and that the Campfire Bore area was, it has classified these areas in reverse. Again, these gold deposits are in the Challenger Gold Deposit region and contain a source that is questionable to our target source from __Geology Forever__. 

In other area the SVM classifier did not perform well, emphasising that the areas of interest are strongly correlated with the distance to faults feature.

### Light GBM and XGBoost

The two tree based boosting algorithms, Light GBM and XGBoost, performed quite similarly and unfortunately provided some bland results. Both models were very vague in their predictions, with their largest probability of mineralisation value being around 60%. This resulted in larger areas with lower probability and does not provide much insight with regards to deposit/camp scale mineralisation predictions.

![Title](https://docs.google.com/uc?export=download&id=1cCSJuJo0u5bRaIbLtpsWPlkFWf0DRb-s)

The LGBM model above shows large, low probability areas of interest (~50%) over the Tarcoola gold region. The predicted areas follow a somewhat similar trend with the distance to faults feature, but also includes a large area considered to be barren. 

![Title](https://docs.google.com/uc?export=download&id=14U3rJyK7wjCDCBdzcIClhRS86rVVueKB)

The XGB model shows a similar large area, low probability response. The above map of the Prominent Hill area shows that almost the entire area is classified as being around 60% likely to host mineralisation. We could have guessed this probability after a few drinks.

These two models potentially provide some insight and could be tweaked to provide better results, but as we are trying to predict individual deposits the current results are not adequate. 

### Convolutional Neural Network

The deep learning model also shows a strong correlation to the distance to faults feature, but is really emphasised in areas of known mineralisation. 

![Title](https://docs.google.com/uc?export=download&id=1arVeioEg4Z1WqbmP3Witjh44PD1dApA9)

The map above shows where the CNN model highest probability lies in respect to the faults feature over the Oak Dam areas. Interestingly the model has picked out the Oak Dam West deposit, but it also has picked out the entire mapped regional fault that its associated with, giving some significant uncertainty on the neural network performance. 

### Random Forest

The random forest model returned some promising visual validation results.

![Title](https://docs.google.com/uc?export=download&id=1agErzSHMdpJ9ghvdXIwSeA6YYSxDhx0y)

In the above map, we can see that the RF model classified some of the more Fe dominant deposits (that contain base metal mineralisation) west of Prominent Hill as being mineralised even though there was no existing data for those areas. There is still quite a spread of mineralisation probability but it is definitely a lot more localised in regards to the models above.

![Title](https://docs.google.com/uc?export=download&id=12LtHX8TkDlqtMhLuxiQMT8fpgIBJzLec)

In the Olympic Dam region, we can see that although the model didn't pick out the Acropolis and Phillips Ridge deposit, the OD and Wirrda Well deposits were defined indicating the possibility of overfitting in our model. Interestingly though the area to the South East of Acropolis showed mineralisation potential and is worth investigating further.

![Title](https://docs.google.com/uc?export=download&id=1UEldew4GpO0NjHmGANqx8CEzfBq7lWbl)

This final RF map shows the Tunkilla Gold deposit region and how the model follows the NW-SE trend of the fault and drilling results. Interestingly, the model includes the Tomahawk South deposit where no data was fed into the model. The area to the west of the Tunkilla deposit is worth investigating.

### K-Nearest Neighbours

Out of all of the models examined in this process, the K-Nearest Neighbours model provided the most promising results, which matches up with the model metrics from __My Friend Robot__. The visual intepretation of the KNN result over the entire craton shows that the model wasn't heavily bias towards any of the features, and results in a combination of all features.

![Title](https://docs.google.com/uc?export=download&id=1lgcp0gESR15ggEaBvUSNq6bBmfhpdZ91)

In the Prominent Hill area, the KNN does a really good job of picking out the surrounding deposits, noticeably Joes Dam West, Cressida, Blue Duck, Proteus and Larissa. These deposits don't explicitly have the model input data associated with them so it's promising that the model has predicted these areas as mineralised on its own.

![Title](https://docs.google.com/uc?export=download&id=1p-1rvGBmeW3Of_NVMYznclcm697GR2Vc)

In the Olympic Dam area we can see that the KNN model has perfectly picked out the Acropolis deposit which is very promising. The Olympic Dam and Wirrda Well areas have also been picked out due to the amount of clustered data points, but there are also scattered areas of interest throughout this map which likely is due to noise in the model. Interestingly, towards the NW of Acropolis we can see another cluster of predicted mineralisation which may be worth investigating.

![Title](https://docs.google.com/uc?export=download&id=1Ko600YTssQXyz9UYyhLfTrUM4TrU3sQn)

Around the Oak Dam area we can see similar scattering of predicted mineralisation, which could be noise in the model or indeed mineralisation potential. The area NE of the Emmie deposits may be worth investigating further, as well as the area to the SW of Horse Well that shows a localised area of increased mineralisation potential.

![Title](https://docs.google.com/uc?export=download&id=1r0WoIOyaE8wJD1i9XBetU2PfjwH_wMGU)

The KNN model showed high mineralisation potential in the Mt Gunson Copper district. This is interesting because there was no data input to the models from this area, and the majority of these deposits are sediment hosted, which does not match up with our mineralisation target from __Geology Forever__. However, it is likely that the source of the base metals deposits in this region were previously hydrothermal/magmatic and have since been remobilised into the overlying sediments. The fact that the model selected these areas as mineralised is promising and is worth investigating.

![Title](https://docs.google.com/uc?export=download&id=1x0JkLZ5CJyObtrsQ-K60olNkbJMbYxuw)

The model also picked up the Bartletts gold deposit in the Tarcoola area without any input data associated with it. It had some issues picking up other deposits in this area, but the fact that this areas was highlighted so distinctively is promising.

![Title](https://docs.google.com/uc?export=download&id=148_NtElRs3V0JyNRZnTm6jxL5LEI7gbc)

Sadly, not one of the models picked up Carapateena or the Fremantle Doctor as an area of interest. This is most likely due to the lack of mapped faults in the area and consequently a higher distance from fault value for these areas.

## Potential Targets
Apart from the localised areas seen in the static maps above, the models return quite sporadic prediction results and without vigourous validation and geological background checking it is easy to assume incorrect mineralisation potential. However, amongst the areas listed above there are two areas of interest we will investigate in more depth

##### 1. Red Lake Copper Area

In the first area, I have defined a drill target of potential mineralisation on the Eastern margin of the Gawler Craton, near the Red Lake Deposit (Expl Tenement 6014). Without delving too deep into the geological history of the region, this area is at intersecting lithological boundaries of the Gawler Range Volcanics (both upper and lower), and the Wallaroo Group. The proposed drillhole (named JM-GC) is situated on a magnetic high, just north of known Red Lake deposit and can be seen in the map below. There are several historic drillholes within the area (drill hole names in yellow, total depths (m) in blue) that we will look at further.

![Title](https://docs.google.com/uc?export=download&id=1-tYCUOisEtDCkubnDvjgx2s0llckmCzZ)

The drillholes __PRL 18__ and __PRL 23__ were drilled in 1978-1982 by Selection Mining Corp, and the more recent __PN-07-09__, __PN-06-03__ and __PN-06-04__ holes were drilled in 2006/2007 by Red Metal/Havilah Resources. These holes did not intersect significicant mineralisation but the deepest hole, __PN-07-09__, did contain localised galena and bornite at approximately 900m depth. There was an increase in copper content with depth in this hole but unfortunately the available lithology log ends at 1000m.

![Title](https://docs.google.com/uc?export=download&id=1xrJST6tTdUlNnNwgtyeASieePDfhbt3K)

The image below shows the disseminated mineralisation style of copper sulphides from 1725-1726m. Although the mineralisation is small, quite deep and doesn't appear to be associated with any local fault/vein systems (from the information available) it is still better than nothing.

![Title](https://docs.google.com/uc?export=download&id=13BBFmwlgtoBpIDsGyfVk-lg-YrEx7OoR)

Hole __PN-06-04__ towards the NE also contains minor copper sulphides, within the Gawler Range Volcanics. The lithology log reported that alteration (albite, calcite) was becoming more intense with depth, but the hole terminated at approximately 600m.

Besides these few holes, this area appears to be very under-explored. It is interesting that the model predicted mineralisation on the northern edge of this magnetic high and the remaining anomaly had a low probability of mineralisation. For this reason I have selected this drillhole location as the first target.

##### 2. McDouall Peak
The second area is located South West of Prominent Hill on the 6069 EL. We will investigate this area further in the final notebook in the  Calculating Copper series, __Extra Extra__.